But : compute the correlation coefficient of the lagged ssh time series between 2005-2018. for GAI-AI

In [1]:
## standart libraries

import os,sys
import numpy as np

# xarray
import xarray as xr

import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib.colors import from_levels_and_colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import cartopy.feature as cfeature
# plot
import regress3d as reg
import cmocean as ccm

In [2]:
from dask.distributed import Client
c = Client()
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 10,Total memory: 20.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44595,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 20.00 GiB
Comm: tcp://127.0.0.1:42425,Total threads: 2
Dashboard: http://127.0.0.1:33743/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:35257,


### prepare the time series to which I am going to compare the 3D matrix : t = 14 (years)

In [3]:
chunk_size = {"x":500,"y":500}
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh_hgr = xr.open_dataset(diri+'mesh_hgr.nc',chunks=chunk_size).squeeze()
tmask = mesh_hgr.tmask

e1t = mesh_hgr.e1t.fillna(0)
e2t = mesh_hgr.e2t.fillna(0)

nav_lon =  mesh_hgr.nav_lon
nav_lat =  mesh_hgr.nav_lat

# constructing the weight
bt = (e1t * e2t).load()# area of each cell

In [4]:
maskBG = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/TRANSPORT_WP1/BGRmaskobs.nc', chunks=chunk_size).BGRmask.squeeze()

diridat = '/gpfswork/rech/cli/uor98hu/MYDATA/SSH/'

daGAI = xr.open_mfdataset(diridat+'GAI/eORCA025.L75-IMHOTEP.GAI_y1980_2018.1m_sossheigCorrected.nc',chunks=chunk_size).sossheig.sel(time_counter=slice("2005","2018"))
daAI = xr.open_mfdataset(diridat+'AI/eORCA025.L75-IMHOTEP.AI_y1980_2018.1m_sossheigCorrected.nc',chunks=chunk_size).sossheig.sel(time_counter=slice("2005","2018"))
daS = xr.open_mfdataset(diridat  +'S/eORCA025.L75-IMHOTEP.S_y1980_2018.1m_sossheigCorrected.nc',chunks=chunk_size).sossheig.sel(time_counter=slice("2005","2018"))

# doing mean over each year
daGAI = daGAI.groupby('time_counter.year').mean('time_counter')
daAI = daAI.groupby('time_counter.year').mean('time_counter')
daS  = daS.groupby('time_counter.year').mean('time_counter')

# constructing the weight
btboxBG = bt.where(maskBG,drop=True)
tmaskboxBG = tmask.where(maskBG,drop=True)
oceanArea = (btboxBG*tmaskboxBG[0])  #tmask : 1 on ocean, 0 on land
totalOceanArea = oceanArea.sum()
WeightboxBG = (oceanArea/totalOceanArea) #.compute()

sshGAIboxBG = (daGAI.where(maskBG,drop=True) * WeightboxBG).sum(('x','y'))
sshAIboxBG  = (daAI.where(maskBG,drop=True) * WeightboxBG).sum(('x','y'))
sshSboxBG   = (daS.where(maskBG,drop=True)  * WeightboxBG).sum(('x','y'))

diffGAI_AI = sshGAIboxBG - sshAIboxBG
diffAI_S = sshAIboxBG - sshSboxBG

In [5]:
# diffGAI_AI.plot()
# diffAI_S.plot()

### définition de la région d'analyse

In [5]:
%%time
chunk_size = {"time_counter":1} # {"x":500,"y":500} "x":360,"y":500


# GAI -AI
# path = "/gpfsscratch/rech/cli/uor98hu/Gr_contribution/integral_wholecolumn"
# S_GR = xr.open_mfdataset(path+"/eORCA025.L75-IMHOTEP.G_y*mall_1m_GrSalinitywholeColumn.nc", chunks=chunk_size).S_GR_integral
# S_GR = S_GR.groupby("time_counter.year").mean('time_counter')
# S_GR_cropped = S_GR.where(nav_lat>=60, drop=True).load()

path = "/gpfsscratch/rech/cli/uor98hu/Gr_contribution/integral_300m"
S_GR = xr.open_mfdataset(path+"/eORCA025.L75-IMHOTEP.G_y*mall_1m_GrSalinity300m.nc", chunks=chunk_size).S_GR_integral
S_GR = S_GR.groupby("time_counter.year").mean('time_counter')
S_GR_cropped = S_GR.where(nav_lat>=60, drop=True).load()

# AI-S
# path = "/gpfsscratch/rech/cli/uor98hu/Rivers_contribution/integral_wholecolumn"
# S_riv = xr.open_mfdataset(path+"/eORCA025.L75-IMHOTEP.RIVERS_y*mall_1m_RivSalinitywholeColumn.nc", chunks=chunk_size).S_riv_integral
# S_riv = S_riv.groupby("time_counter.year").mean('time_counter') *(-1)
# S_riv_cropped = S_riv.where(nav_lat>=60, drop=True).load()

path = "/gpfsscratch/rech/cli/uor98hu/Rivers_contribution/integral_300m"
S_riv = xr.open_mfdataset(path+"/eORCA025.L75-IMHOTEP.RIVERS_y*mall_1m_RivSalinity300m.nc", chunks=chunk_size).S_riv_integral
S_riv = S_riv.groupby("time_counter.year").mean('time_counter')
S_riv_cropped = S_riv.where(nav_lat>=60, drop=True).load()

CPU times: user 6.96 s, sys: 768 ms, total: 7.73 s
Wall time: 11.5 s


# compute the correlation

### GAI - AI

In [6]:
%%time
# compute lagged correlation year per year
# 2005-2018 time period is 14 years long
# we go back up to 10 years of lagged correlation

# intialisation
r_matrix_gr = np.zeros((25,232,1221))
expvar,coeffs,p,r = reg.regress3d(diffGAI_AI.values,S_GR_cropped.values[-14:,:,:])
r_matrix_gr[0,:,:] = r

for lag in np.arange(1,25):
    istart = 14+lag
    expvar,coeffs,p,r = reg.regress3d(diffGAI_AI.values,S_GR_cropped.values[-istart:-lag,:,:])
    r_matrix_gr[lag,:,:] = r
    
r_matrix2plot_Gr = xr.DataArray(r_matrix_gr,dims=["lag","y", "x"], coords=dict(lag=np.arange(0,25)))

/gpfsdswork/projects/rech/cli/uor98hu/pythonfiles/lagged_correlation/regress3d.py:53: RuntimeWarning: invalid value encountered in true_divide
  expvar=(1-(SSE/SSY))*100.0
/gpfsdswork/projects/rech/cli/uor98hu/pythonfiles/lagged_correlation/regress3d.py:72: RuntimeWarning: invalid value encountered in true_divide
  t = coeffs[1,:] / serr
/gpfsdswork/projects/rech/cli/uor98hu/pythonfiles/lagged_correlation/regress3d.py:80: RuntimeWarning: invalid value encountered in sqrt
  r=0.1*np.sqrt(expvar)


CPU times: user 29.1 s, sys: 18 s, total: 47.1 s
Wall time: 28.4 s


### AI -S

In [7]:
%%time
# compute lagged correlation year per year
# 2005-2018 time period is 14 years long
# we go back up to 10 years of lagged correlation

# intialisation
r_matrix_riv = np.zeros((25,232,1221))
expvar,coeffs,p,r = reg.regress3d(diffAI_S.values,S_riv_cropped.values[-14:,:,:])
r_matrix_riv[0,:,:] = r

for lag in np.arange(1,25):
    istart = 14+lag
    expvar,coeffs,p,r = reg.regress3d(diffAI_S.values,S_riv_cropped.values[-istart:-lag,:,:])
    r_matrix_riv[lag,:,:] = r
    
r_matrix2plot_riv = xr.DataArray(r_matrix_riv,dims=["lag","y", "x"], coords=dict(lag=np.arange(0,25)))

CPU times: user 28.8 s, sys: 18.2 s, total: 47 s
Wall time: 28.6 s


# comparaison Gr et Riv

In [8]:
%%time
comp_matrix = np.zeros((25,232,1221))

for lag in np.arange(25):
    for y in np.arange(232):
        for x in np.arange(1221):
            if r_matrix_riv[lag,y,x] * r_matrix_gr[lag,y,x] >0:
                comp_matrix[lag,y,x] = (r_matrix_riv[lag,y,x] + r_matrix_gr[lag,y,x])/2
            else :
                comp_matrix[lag,y,x] = np.nan
                

CPU times: user 11 s, sys: 103 ms, total: 11.1 s
Wall time: 10.9 s


In [9]:
comp_matrix2plot = xr.DataArray(comp_matrix,dims=["lag","y", "x"], coords=dict(lag=np.arange(0,25)))

In [8]:
# lag = 16
# istart = 14+lag

# # S_GR_cropped[-istart:-lag,:,:]
# S_riv_cropped[-istart:-lag,:,:]

# # r_matrix2plot

In [45]:
# old method
# lag = 0
# expvar,coeffs,p,r = reg.regress3d(diffGAI_AI.values,S_GR_cropped.values[-14:,:,:])
# # lag = 5 ans
# expvar,coeffs,p,r_5 = reg.regress3d(diffGAI_AI.values[5:],S_GR_cropped.values[:-5,:,:]) 
# # lag = 10 ans
# expvar,coeffs,p,r_10 = reg.regress3d(diffGAI_AI.values[10:],S_GR_cropped.values[:-10,:,:]) 
# # lag = 15
# expvar,coeffs,p,r_15 = reg.regress3d(diffGAI_AI.values[15:],S_GR_cropped.values[:-15,:,:]) 
# # lag =20
# expvar,coeffs,p,r_20 = reg.regress3d(diffGAI_AI.values[20:],S_GR_cropped.values[:-20,:,:]) 
# # lag = 25
# expvar,coeffs,p,r_25 = reg.regress3d(diffGAI_AI.values[25:],S_GR_cropped.values[:-25,:,:]) 

# r2plot = xr.DataArray(r,dims=["y", "x"])
# r_matrix[0,:,:] = r
# r_matrix[1,:,:] = r_5
# r_matrix[2,:,:] = r_10
# r_matrix[3,:,:] = r_15
# r_matrix[4,:,:] = r_20
# r_matrix[5,:,:] = r_25

# r_matrix2plot = xr.DataArray(r_matrix,dims=["lag","y", "x"], coords=dict(lag=np.arange(0,30,5)))

# plot nicely

In [10]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [11]:
# read grid files
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
nav_lat = xr.open_dataset(diri+'mesh_hgr.nc')['nav_lat'].squeeze()#.where(nav_lat>=60, drop=True)

maskBG2plot = maskBG.where(nav_lat>=60, drop=True)
tmask = xr.open_dataset(diri+'mesh_hgr.nc')['tmask'][0,0,:,:].where(nav_lat>=60, drop=True) # grille lat/lon
nav_lon = xr.open_dataset(diri+'mesh_hgr.nc')['nav_lon'].squeeze().where(nav_lat>=60, drop=True)
mbathy = xr.open_dataset(diri+'eORCA025.L75_domain_cfg_closed_seas_greenland.nc')['bathy_meter'].isel(t=0).where(nav_lat>=60, drop=True) # grille lat/lon
nav_lat = nav_lat.where(nav_lat>=60, drop=True)

# land mask from model
m2plt      = tmask.where(tmask==0)

# default gridlines parameters
gridl=False
incrgridlon=30
incrgridlat=10

# vue N.Atl
# loncentr=330 # before it was zero
# latcentr=70
#vue polaire
loncentr=200 # before it was zero
latcentr=90

sath=2085831

nav_lon2plot = nav_lon.fillna(0)
nav_lat2plot = nav_lat.fillna(0)


In [12]:
cmap = ccm.cm.balance
my_map1 = truncate_colormap(cmap, 0, 0.4)
my_map2 = truncate_colormap(cmap, 0.6, 1)
colors1 = my_map1(np.linspace(0., 1, 50))
colors2 = my_map2(np.linspace(0., 1, 50))
colors = np.vstack(( colors1, colors2))
mycm = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)

# plot correlation maps

In [10]:
lmin = -1.0
incr = 0.1
lmax = lmin*(-1)+0.1
# define the bins and normalize
bounds = np.arange(lmin,lmax,incr)
norm = mcolors.BoundaryNorm(bounds, mycm.N)

In [11]:
#========= CREATE FIGURE
diro = "/gpfsscratch/rech/cli/uor98hu/corr_maps_gr/" # GAI -AI
# diro = "/gpfsscratch/rech/cli/uor98hu/corr_maps_riv/" # AI - S

r_matrix2plot = r_matrix2plot_Gr
# RUN to PLOT
for i in np.arange(25):
    data2plot = r_matrix2plot.isel(lag=i).where(tmask!=0,0).squeeze()
    # yr, mm = dates(S_GR_short.isel(time_counter=t))
    tlabel= "Correlation coefficient for time lag = "+str(r_matrix2plot.lag[i].values)+" years" #"whole column S_GR integral for "+str(yr)+"-"+f"{mm:02d}" + r" [10$-3$]"
    namo  ="GAI_AI_corr_map_SSH_S_GR_lag"+str(r_matrix2plot.lag[i].values)+"y.png"
    # namo  ="AI_S_corr_map_SSH_S_riv_lag"+str(r_matrix2plot.lag[i].values)+"yv2.png"
    print(namo)
    
    #===========================================
    fig = plt.figure(figsize=([18,10]),facecolor='none', dpi=72*2)

    trdata  = ccrs.PlateCarree() # Data system proj (if coords are in lat lon, use PlateCarre here)
    ax = plt.axes(projection= ccrs.NearsidePerspective(central_longitude=loncentr, central_latitude=latcentr,satellite_height=sath))

    #========= PLOT DATA

    ax.contour(nav_lon,nav_lat,maskBG2plot,1,colors="k",transform=trdata)
    cs  = plt.pcolormesh(nav_lon2plot, nav_lat2plot, data2plot,shading='flat',cmap = mycm,transform=trdata, norm=norm)
    fig.colorbar(cs,extend='neither', ticks=bounds,label=r'correlation coefficient')

    # cs2  = plt.pcolormesh(nav_lon, nav_lat, m2plt, shading='flat',transform=trdata) 
    #========= coastlines and gridlines
    rivers = cartopy.feature.NaturalEarthFeature(category='physical', name='rivers_lake_centerlines',scale='50m',facecolor='none',edgecolor='b')
    clr2 = ax.add_feature(rivers,alpha=0.7,facecolor='none',edgecolor='blue',zorder=6)
    cl2 = ax.add_feature(cfeature.LAND.with_scale('50m'),facecolor="dimgrey",edgecolor= 'none',alpha=1,zorder=5)
    gl =ax.gridlines(xlocs=range(-180,181,incrgridlon), ylocs=range(-90,91,incrgridlat),draw_labels=True,linewidth=1, color='#585858', alpha=0.3, linestyle='--',zorder=8)
    label_style = {'size': 12, 'color': '#BDBDBD', 'weight': 'normal'}
    gl.xlabel_style = label_style; gl.ylabel_style = label_style


    #============= BATHYMETRY
    bathy_levels = [200,1000,2000,4000] # We will apply a Gaussian filter to smooth the data. 
    color_levels= ['k','navy','mediumblue','slateblue']
    # west
    Ct1 = ax.contour(nav_lon,nav_lat,mbathy.where(nav_lon<0), bathy_levels,colors=color_levels,linewidths=0.7,linestyles='solid',transform=trdata)
    # east
    Ct2 = ax.contour(nav_lon,nav_lat,mbathy.where(nav_lon>0), bathy_levels,colors=color_levels,linewidths=0.7,linestyles='solid',transform=trdata)


    plt.title(tlabel,fontsize=24)
    # plt.show()
    fig.savefig(diro+namo, facecolor=fig.get_facecolor(), bbox_inches='tight') #,edgecolor='none', 
    # plt.close(fig)
    plt.clf()

GAI_AI_corr_map_SSH_S_GR_lag0y.png


/gpfslocalsup/pub/anaconda-py3/2020.11/envs/climate_science/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:1597: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  X, Y, C, shading = self._pcolorargs('pcolormesh', *args,


GAI_AI_corr_map_SSH_S_GR_lag1y.png


/gpfslocalsup/pub/anaconda-py3/2020.11/envs/climate_science/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:1597: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  X, Y, C, shading = self._pcolorargs('pcolormesh', *args,


GAI_AI_corr_map_SSH_S_GR_lag2y.png


...

<Figure size 2592x1440 with 0 Axes>

<Figure size 2592x1440 with 0 Axes>

<Figure size 2592x1440 with 0 Axes>

<Figure size 2592x1440 with 0 Axes>

<Figure size 2592x1440 with 0 Axes>

# plot comparison map

In [13]:
lmin = -1.0
incr = 0.1
lmax = lmin*(-1)+0.1
# define the bins and normalize
bounds = np.arange(lmin,lmax,incr)
norm = mcolors.BoundaryNorm(bounds, mycm.N)

In [14]:
#========= CREATE FIGURE
# diro = "/gpfsscratch/rech/cli/uor98hu/corr_maps/" # GAI -AI
# diro = "/gpfsscratch/rech/cli/uor98hu/corr_maps_riv/" # AI - S
diro = "/gpfsscratch/rech/cli/uor98hu/comp_corr_maps/" # comparison maps


# RUN to PLOT
for i in np.arange(10):
    data2plot = comp_matrix2plot.isel(lag=i).where(tmask!=0,0).squeeze()
    tlabel= "Comparison correlation coefficient for time lag = "+str(comp_matrix2plot.lag[i].values)+" years" #"whole column S_GR integral for "+str(yr)+"-"+f"{mm:02d}" + r" [10$-3$]"
    namo  ="comp_corr_map_SSH_S_anom_300_lag"+str(comp_matrix2plot.lag[i].values)+"y.png"
    # namo  ="comp_corr_map_SSH_S_anom_bottom_lag"+str(comp_matrix2plot.lag[i].values)+"y.png"

    print(namo)
    
    #===========================================
    fig = plt.figure(figsize=([18,10]),facecolor='none', dpi=72*2)

    trdata  = ccrs.PlateCarree() # Data system proj (if coords are in lat lon, use PlateCarre here)
    ax = plt.axes(projection= ccrs.NearsidePerspective(central_longitude=loncentr, central_latitude=latcentr,satellite_height=sath))

    #========= PLOT DATA

    ax.contour(nav_lon,nav_lat,maskBG2plot,1,colors="k",transform=trdata)
    cs  = plt.pcolormesh(nav_lon2plot, nav_lat2plot, data2plot,shading='flat',cmap = mycm,transform=trdata, norm=norm)
    fig.colorbar(cs,extend='neither', ticks=bounds,label=r'correlation coefficient')

    ct = ax.contour(nav_lon,nav_lat,data2plot,[-0.8,0.8],colors=['white','white'],linewidths=0.7,linestyles='dashed',transform=trdata)

    # cs2  = plt.pcolormesh(nav_lon, nav_lat, m2plt, shading='flat',transform=trdata) 
    #========= coastlines and gridlines
    rivers = cartopy.feature.NaturalEarthFeature(category='physical', name='rivers_lake_centerlines',scale='50m',facecolor='none',edgecolor='b')
    clr2 = ax.add_feature(rivers,alpha=0.7,facecolor='none',edgecolor='blue',zorder=6)
    cl2 = ax.add_feature(cfeature.LAND.with_scale('50m'),facecolor="dimgrey",edgecolor= 'none',alpha=1,zorder=5)
    gl =ax.gridlines(xlocs=range(-180,181,incrgridlon), ylocs=range(-90,91,incrgridlat),draw_labels=True,linewidth=1, color='#585858', alpha=0.3, linestyle='--',zorder=8)
    label_style = {'size': 12, 'color': '#BDBDBD', 'weight': 'normal'}
    gl.xlabel_style = label_style; gl.ylabel_style = label_style


    #============= BATHYMETRY
    bathy_levels = [200,1000,2000,4000] # We will apply a Gaussian filter to smooth the data. 
    color_levels= ['k','navy','mediumblue','slateblue']
    # west
    Ct1 = ax.contour(nav_lon,nav_lat,mbathy.where(nav_lon<0), bathy_levels,colors=color_levels,linewidths=0.7,linestyles='solid',transform=trdata)
    # east
    Ct2 = ax.contour(nav_lon,nav_lat,mbathy.where(nav_lon>0), bathy_levels,colors=color_levels,linewidths=0.7,linestyles='solid',transform=trdata)


    plt.title(tlabel,fontsize=24)
    # plt.show()
    fig.savefig(diro+namo, facecolor=fig.get_facecolor(), bbox_inches='tight') #,edgecolor='none', 
    # plt.close(fig)
    plt.clf()

comp_corr_map_SSH_S_anom_300_lag0y.png


/gpfslocalsup/pub/anaconda-py3/2020.11/envs/climate_science/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:1597: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  X, Y, C, shading = self._pcolorargs('pcolormesh', *args,


comp_corr_map_SSH_S_anom_300_lag1y.png


/gpfslocalsup/pub/anaconda-py3/2020.11/envs/climate_science/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:1597: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  X, Y, C, shading = self._pcolorargs('pcolormesh', *args,


comp_corr_map_SSH_S_anom_300_lag2y.png


...

<Figure size 2592x1440 with 0 Axes>

<Figure size 2592x1440 with 0 Axes>

<Figure size 2592x1440 with 0 Axes>

<Figure size 2592x1440 with 0 Axes>

<Figure size 2592x1440 with 0 Axes>